In [ ]:
import json
import csv
import pyperclip
import sys
sys.path.insert(1, './scr')
from PONS import PONS_writer, PONS_refiner, PONS_reader
from toolbox import AnkiCardWriter

def read_csv_to_list(file_path:str):
    word_list = []
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            # Assuming each row contains only one word
            word_list.append(row[0])
    return word_list

## Configuration

In [ ]:
today = '20240426'

words_to_look_up = read_csv_to_list('Begegnungen.csv')
print(f'There are {len(words_to_look_up)} words.')

## Add new words

### Look up the words in the PONS dictionary and export the results

In [ ]:
writer = PONS_writer()
writer.look_up(word_list=words_to_look_up)

### Generate the prompt to generate example sentences

In [ ]:
start_idx = 11
end_idx = 20
words_to_refine = words_to_look_up[start_idx-1:end_idx]
refiner = PONS_refiner()
refiner.generate_eg_prompt(words_to_refine)

### Paste the response with example sentences here and generate the prompt for a refined JSON file

In [ ]:
ai_response_str_eg = pyperclip.paste()
ai_response_dict_eg = json.loads(ai_response_str_eg)
refiner.generate_def_prompt(word_list=words_to_refine, temp_dict=ai_response_dict_eg)

### Paste the response with the refined JSON string here

In [ ]:
ai_response_str_def = pyperclip.paste()
ai_response_dict_def = json.loads(ai_response_str_def)
refiner.import_refined_dict(ai_response_dict_def)
refiner.finish_import()

## Generate Anki cards

In [ ]:
reader = PONS_reader(word_list=words_to_look_up)
anki_writer = AnkiCardWriter(word_entry_list=reader.get_word_entry_list())
anki_writer.write_cards(csv_path=f'{today}.csv')